In [4]:
import requests
from bs4 import BeautifulSoup

# 実行フロー
1. ページングページURLリストを作成。
2. 記事URLリストを取得・保存。(記事内ページングを考慮)
3. 記事URLリストからHTMLをそのまま取得。(URLと一緒に)
4. 取得したHTMLから本文・タイトル・日付を抽出。

In [108]:
"""
http://number.bunshun.jp/category/column?page=900
https://victorysportsnews.com/?p=630
http://www.saga-s.co.jp/category/news-original/%E3%82%B9%E3%83%9D%E3%83%BC%E3%83%84?page=610
https://sports.yahoo.co.jp/column/list?type=sportsnavi&p=1250
https://azrena.com/post/category/article/all/page/50/
https://www.shimotsuke.co.jp/category/news-original/%E3%82%B9%E3%83%9D%E3%83%BC%E3%83%84?page=90
https://www.chibanippo.co.jp/newspack/sports?page=2763
http://www.kanaloco.jp/sports/generalsports/298/
http://www.isenp.co.jp/category/sports/page/56/
http://www.topics.or.jp/category/news-original/%E3%82%B9%E3%83%9D%E3%83%BC%E3%83%84?page=180
https://www.nishinippon.co.jp/nsp/archive/?page=1063
"""

import requests
from bs4 import BeautifulSoup
import re

class UrlGetter:
    def __init__(self):
        self.max_pages = [900, 630, 610, 1250, 50, 90, 2763, 298, 56, 180, 1063]
    
    def get_0(self, page):
        return "http://number.bunshun.jp/category/column?page={}".format(page)
    
    def get_1(self, page):
        return "https://victorysportsnews.com/?p={}".format(page)
    
    def get_2(self, page):
        return "http://www.saga-s.co.jp/category/news-original/%E3%82%B9%E3%83%9D%E3%83%BC%E3%83%84?page={}".format(page)

    def get_3(self, page):
        return "https://sports.yahoo.co.jp/column/list?type=sportsnavi&p={}".format(page)
    
    def get_4(self, page):
        return "https://azrena.com/post/category/article/all/page/{}/".format(page)
    
    def get_5(self, page):
        return "https://www.shimotsuke.co.jp/category/news-original/%E3%82%B9%E3%83%9D%E3%83%BC%E3%83%84?page={}".format(page)
    
    def get_6(self, page):
        return "https://www.chibanippo.co.jp/newspack/sports?page={}".format(page)
    
    def get_7(self, page):
        return "http://www.kanaloco.jp/sports/generalsports/{}/".format(page)
    
    def get_8(self, page):
        return "http://www.isenp.co.jp/category/sports/page/{}/".format(page)
    
    def get_9(self, page):
        return "https://www.nishinippon.co.jp/nsp/archive/?page={}".format(page)
    
    def get_10(self, page):
        return "http://www.topics.or.jp/category/news-original/%E3%82%B9%E3%83%9D%E3%83%BC%E3%83%84?page={}".format(page)
        
    def get_url(self, site_num, *params):
        return getattr(self, "get_{}".format(site_num))(*params)

In [78]:
class ArticleGetter:
    def __init__(self, base_url):
        self.base_url = base_url
        r = requests.get(base_url)
        self.soup = BeautifulSoup(r.content, "html.parser")
    
    def get_0(self):
        section = self.soup.find("section", {"class": "category-archive"})
        return list(set(["http://number.bunshun.jp"+result['href'] for result in section.find_all("a") if result['href'].startswith("/articles/")]))

    def get_1(self):
        section = self.soup.find("div", {"class":"article-list-page"})
        return list(set(["https://victorysportsnews.com"+result['href']+"/original" for result in section.find_all("a", {"class":"link-card-inner"})]))
    
    def get_2(self):
        section = self.soup.find("div", {"id": "article-category-list"})
        return list(set("http://www.saga-s.co.jp" + result['href'] for result in section.find_all("a") if result['href'].startswith("/articles/")))
    
    def get_3(self):
        section = self.soup.find_all("p", {"class":"articleTitle"})
        return list(set(result.find("a")['href'] for result in section))
    
    def get_4(self):
        return list(set(result['href'] for result in self.soup.find_all("a", {"class": "panel"})))
    
    def get_5(self):
        section = self.soup.find("ul", {"class": "article-category-list"})
        return list(set("https://www.shimotsuke.co.jp" + result['href'] for result in section.find_all("a") if result['href'].startswith("/articles/")))
    
    def get_6(self):
        section = self.soup.find("div", {"class": "item-list"})
        return list(set("https://www.chibanippo.co.jp" + result['href'] for result in section.find_all("a")))
    
    def get_7(self):
        section = self.soup.find("ul", {"class": "post-grid"})
        return list(set("http://www.kanaloco.jp" + result['href'] for result in section.find_all("a")))
    
    def get_8(self):
        section = self.soup.find_all("h3", {"class": "entry-title"})
        return list(set(result.find("a")['href'] for result in section))
        
    def get_9(self):
        section = self.soup.find("ul", {"class": "NP-articleList"})
        return list(set("https://www.nishinippon.co.jp"+result['href'] for result in section.find_all("a")))
    
    def get_10(self):
        section = self.soup.find("div", {"id": "article-category-list"})
        return list(set("http://www.topics.or.jp" + result['href'] for result in section.find_all("a") if result['href'].startswith("/articles/")))
    
    def get_articles(self, site_num):
        return getattr(self, "get_{}".format(site_num))()

In [245]:
site_num = 3
urlgetter = UrlGetter()
base_url = urlgetter.get_url(site_num, 1)
ArticleGetter(base_url).get_articles(site_num)

['https://sports.yahoo.co.jp/column/detail/201807220002-spnavi',
 'https://2020.yahoo.co.jp/column/detail/201807170006-spnavi',
 'https://sports.yahoo.co.jp/column/detail/201807220001-spnavi',
 'https://sports.yahoo.co.jp/column/detail/201807200005-spnavi',
 'https://sports.yahoo.co.jp/column/detail/201807210007-spnavi',
 'https://sports.yahoo.co.jp/column/detail/201807230003-spnavi',
 'https://sports.yahoo.co.jp/column/detail/201807200008-spnavi',
 'https://sports.yahoo.co.jp/column/detail/201807210003-spnavi',
 'https://sports.yahoo.co.jp/column/detail/201807210001-spnavi',
 'https://sports.yahoo.co.jp/column/detail/201807230002-spnavi',
 'https://sports.yahoo.co.jp/column/detail/201807200002-spnavi',
 'https://sports.yahoo.co.jp/column/detail/201807210004-spnavi',
 'https://sports.yahoo.co.jp/column/detail/201807210002-spnavi',
 'https://sports.yahoo.co.jp/column/detail/201807210005-spnavi',
 'https://sports.yahoo.co.jp/column/detail/201807020002-spnavi',
 'https://sports.yahoo.co.j

In [229]:
import re
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from urllib.parse import urlparse

class ArticleExtractor:
    def __init__(self, target_url):
        self.target_url = target_url
        self.driver = webdriver.PhantomJS()
        self.driver.get(target_url)
        self.soup = BeautifulSoup(self.driver.page_source, "html.parser")
        [s.extract() for s in self.soup('script')]
        [s.extract() for s in self.soup('link')]
    
    def extract_0(self):
        tmp = self.soup.find("div", {"class": "list-pagination-02"})
        inner_pages = ["http://number.bunshun.jp" + d['href'] for d in tmp.find_all("a")]
        title = self.soup.find("h2").get_text()
        day = self.soup.find("p", {"class": "posted"})
        day.find("span").extract()
        day = day.get_text()
        body = self.soup.find("div", {"class": "single"}).get_text()
        for page in inner_pages:
            tmp_soup = BeautifulSoup(requests.get(page).content, "html.parser")
            [s.extract() for s in tmp_soup('script')]
            [s.extract() for s in tmp_soup('link')]
            body += tmp_soup.find("div", {"class": "single"}).get_text()
        return title, body, day
    
    def extract_1(self):
        headline = self.soup.find("div", {"class": "article-headline"})
        title = headline.find("h1").get_text()
        day = headline.find("time").get_text()
        body = headline.find("p").get_text()
        abody = self.soup.find("div", {"class": "article-body"})
        abody.find('div', {"class": "author-status"}).extract()
        [s.extract() for s in abody.find_all("a")]
        body += ''.join([d.get_text() for d in abody.find_all("p")])
        return title, body, day
    
    def extract_2(self):
        title = self.soup.find("h1").get_text()
        day = self.soup.find("div", {"class": "pubdate"}).get_text()
        abody = self.soup.find("div", {"class": "article-body"})
        body = ''.join(d.get_text() for d in abody.find_all("p"))
        return title, body, day
    
    def extract_3(self):
        title = self.soup.find("div", {"class":"columnTitle"}).get_text()
        day = self.soup.find("li", {"class":"postDate"}).get_text()
        body = self.soup.find("div", {"class":"mainSection"}).get_text()
        parsed_uri = urlparse(self.target_url)
        top_url = '{uri.scheme}://{uri.netloc}'.format(uri=parsed_uri)
        inner_pages = list(set([top_url + d['href'] for d in self.soup.find("div", {"class":"navPage"}).find_all("a")]))
        for page in inner_pages:
            self.driver.get(page)
            tmp_soup = BeautifulSoup(self.driver.page_source, "html.parser")
            [s.extract() for s in tmp_soup('script')]
            [s.extract() for s in tmp_soup('link')]
            body += tmp_soup.find("div", {"class": "mainSection"}).get_text()
        return title, body, day
    
    def extract_4(self):
        title = self.soup.find("h1", {"class": "title"}).get_text()
        day = self.soup.find("span", {"class": "date"}).get_text()
        content = self.soup.find("div", {"class": "m-article-content"})
        body = ''.join([p.get_text() for p in content.find_all("p")])
        body = re.sub('<[^<]+?>', '', body)
        return title, body, day
    
    def extract_5(self):
        title = self.soup.find("h1", {"class": "title"}).get_text()
        day = self.soup.find("div", {"class": "pubdate"}).get_text()
        content = self.soup.find("article")
        content.find("div", {"class": "smart-link"}).extract()
        body = ''.join([d.get_text() for d in content.find_all("p")])
        body = re.sub('<[^<]+?>', '', body)
        return title, body, day
    
    def extract_6(self):
        title = self.soup.find("h1", {"class": "title"}).get_text()
        day = self.soup.find("div", {"class": "submitted"}).get_text()
        content = self.soup.find("article")
        body = ''.join([d.get_text() for d in content.find_all("p")])
        body = re.sub('<[^<]+?>', '', body)
        return title, body, day
    
    def extract_7(self):
        title = self.soup.find("h1", {"id": "post-title"}).get_text()
        day = self.soup.find("li", {"class": "date"}).get_text()
        body = self.soup.find("div", {"id": "post-content"}).get_text()
        body = re.sub('<[^<]+?>', '', body)
        return title, body, day
    
    def extract_8(self):
        title = self.soup.find("h1", {"class": "entry-title"}).get_text()
        day = self.soup.find("span", {"class": "entry-meta-date"}).get_text()
        content = self.soup.find("div", {"class": "entry-content"})
        body = ''.join([d.get_text() for d in content.find_all("p")])
        return title, body, day
    
    def extract_9(self):
        title = self.soup.find("h1").get_text()
        day = self.soup.find("time").get_text()
        body = self.soup.find("div", {"class": "NP-articleText"}).get_text()
        body = re.sub('<[^<]+?>', '', body)
        return title, body, day
    
    def extract_10(self):
        title = self.soup.find("h1", {"class": "title"}).get_text()
        day = self.soup.find("div", {"class": "pubdate"}).get_text()
        body = self.soup.find("div", {"class": "article-body"}).get_text()
        body = re.sub('<[^<]+?>', '', body)
        return title, body, day
    
    def extract_article(self, site_num):
        title, body, day = getattr(self, "extract_{}".format(site_num))()
        title = title.replace("\n", " ").replace("\u3000", " ").replace("\r","").strip()
        body = body.replace("\n", " ").replace("\u3000", " ").replace("\r", "").strip()
        body = body.replace("\xa0", "")
        day = day.replace("\n", " ").replace("\u3000", " ").replace("\r", "").strip()
        return title, body, day

In [234]:
site_type = 10
urlgetter = UrlGetter()
base_url = urlgetter.get_url(site_type, 1)
target_url = ArticleGetter(base_url).get_articles(site_type)[3]
ArticleExtractor(target_url).extract_article(site_type)

/opt/conda/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


('徳島ヴォルティス、大宮に２－１で競り勝つ 前川先制、岩尾が決勝弾',
 '拡大する   前半、シュートを放つ徳島のＦＷウタカ（中央）＝鳴門ポカリスエットスタジアム                サッカーＪ２徳島ヴォルティスは２１日、ホーム・鳴門ポカリスエットスタジアムで大宮アルディージャと対戦し、後半２ゴールを挙げて２ー１で競り勝った。リーグ戦で９試合負けなしだった大宮に黒星をつけた。徳島は９勝４分け１１敗で、勝ち点を３１とした。順位は１５位。  徳島は新加入のＦＷピーター・ウタカが先発した。前半、序盤は徳島がボールを保持してチャンスをつくったが、中盤になると、大宮にボールを持たれ、セットプレーからゴールに迫られるなど、１５分ごとに攻撃の主導権が入れ替わる展開。２９分には、大宮に中盤でパスカットされてカウンターを受けたが、シュートはポストに当たり失点を免れた。  その後、徳島が反撃。３０分、島屋からのボールをウタカが収め、パスを受けた前川大河がゴール左のニアサイドを狙ってシュート。しかし、ＧＫの好守でネットは揺らせなかった。前川は３６分にもミドルで狙ったが枠を捉えられなかった。終盤にはウタカがペナルティエリア内で切り返してシュートを放つも阻まれ、直後のコーナーキックのヘディングシュートも枠をそれ、０－０で前半を終えた。  後半はめまぐるしく攻守が入れ替わる展開だったが、１１分、左サイドの岩尾憲が送ったパスに前川大河が抜けだし、左足で逆サイドのネットにボールを流し込んで先制した。直後に、大宮のＧＫから始まったカウンターを防ぎきれずに同点とされたが、２１分にウタカからのパスを受けた岩尾がシュートを決めて勝ち越した。その後、左サイドの大本祐槻の積極的な攻め上がりや前線のウタカがチャンスをつくるなど攻勢を保った。  大宮が前線に長身の選手を投入するなど攻撃の圧力を強め、押し込まれる時間帯もあったが、守備陣が粘り強くはね返し、１点のリードを守り切った。 ◇明治安田Ｊ２第２４節  鳴門ポカリスエットスタジアム  観衆５８０１人 徳島 ２－１ 大宮    ０－０    ２－１ 【得点者】徳島＝前川大河（後半１１分）岩尾憲（後半２１分）      大宮＝大前元紀（後半１４分）',
 '7/21  21:30')

In [246]:
def detect_sitenum(url):
    if "number.bunshun.jp" in url:
        return 0
    elif "victorysportsnews.com" in url:
        return 1
    elif "www.saga-s.co.jp" in url:
        return 2
    elif "azrena.com" in url:
        return 4
    elif "www.shimotsuke.co.jp" in url:
        return 5
    elif "www.chibanippo.co.jp" in url:
        return 6
    elif "www.kanaloco.jp" in url:
        return 7
    elif "www.isenp.co.jp" in url:
        return 8
    elif "www.nishinippon.co.jp" in url:
        return 9
    elif "www.topics.or.jp" in url:
        return 10
    elif "yahoo.co.jp" in url:
        return 3
    else:
        return None
        

In [247]:
def read_urllist(filename):
    with open(filename, "r") as f:
        urllist = f.readlines()
    return urllist

urllist = read_urllist("urllist.txt")

In [248]:
from tqdm import tqdm
with open("extracted_data.csv", "a") as f:
    f.write("url,title,body,day\n")
    for url in tqdm(urllist):
        try:
            title, body, day = ArticleExtractor(url).extract_article(detect_sitenum(url))
            f.write("{},{},{},{}".format(url,title,body,day))
        except:
            print(url)

156975